In [66]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymongo
from selenium.common.exceptions import NoSuchElementException
from cost_of_living import *
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
from hidden import debt
from math import cos, sqrt

In [ ]:
data = pd.read_excel('data/citydf.xlsx')
data['city'].replace('Washington', "Washington D.C.", inplace=True)

### Check Data for missing values --> Which appear to be denoted as '?'

In [20]:
data[data['price'] == '?']

,category,city,item,price
28,Transportation,Palo-Alto,Monthly Pass (Regular Price),?
796,Markets,Mountain-View,Cigarettes 20 Pack (Marlboro),?
1240,Transportation,Irvine,Taxi 1 mile (Normal Tariff),?
1515,Transportation,Plano,Taxi 1 mile (Normal Tariff),?
1516,Transportation,Plano,Taxi 1hour Waiting (Normal Tariff),?


### Write function to find lat_long of a city

In [32]:
from geopy.geocoders import Nominatim
def find_lat_long(city):
    geolocator = Nominatim(user_agent="LifestyleDesign")
    location = geolocator.geocode(city)
    return location[1]

In [ ]:
data['lat_long'] = data['city'].apply(find_lat_long)

### For efficieny, we'll calculate the lat_long once for each city and map that value to each city

In [54]:
cities = data.city.unique()

In [62]:
city_lat_long = {}
for city in cities:
    city_lat_long[city] = find_lat_long(city)

In [64]:
data["lat_long"] = data["city"].map(city_lat_long)

### Now, we'll write a function to find the closest city to each city (using the lat_long distance). This can POTENTIALLY allow us to fill in missing values.

In [136]:
from math import cos, asin, sqrt

def find_distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest_city(lat_long):
    closest = {}
    for k,v in city_lat_long.items():
        if v == lat_long:
            pass
        else:
            distance = find_distance(lat_long[0], lat_long[1], v[0], v[1])
            closest[k] = distance
    return min(closest.keys(), key=closest.__getitem__)

In [137]:
data.loc[0][4]

(37.4455862, -122.1619289)

In [138]:
closest_city(data.loc[0][4])

'Mountain-View'

### Create a dict of each city and it's closest city

In [148]:
city_neighbor = {}
for city, lat_long in city_lat_long.items():
    city_neighbor[city] = closest_city(lat_long)

### Map neighbors to city in dataframe

In [150]:
data["city_neighbor"] = data["city"].map(city_neighbor)

### Begin writing function to replace missing prices with city neighbors

In [207]:
def find_neighbor_price(city, item):
    neighbor = city_neighbor[city]
    price = data.loc[(data['city'] == neighbor) & data['item'].isin([item])]['price'].values[0]
    return (f"Neighbor: {neighbor}, Price: {price}")

In [208]:
find_neighbor_price('Palo-Alto', 'Monthly Pass (Regular Price)')

'Neighbor: Mountain-View, Price: 70.00'

## Write a function to produce cost-of-living in multiple cities

In [239]:
monthly_constants = {
'Cinema, International Release, 1 Seat' : 2,
'Fitness Club, Monthly Fee for 1 Adult' : 2,
'Basic (Electricity, Heating, Cooling, Water, Garbage) for 915 sq ft Apartment' : 1,
'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)' : 1,
'Gasoline (1 gallon)' : 15,
'Imported Beer (11.2 oz small bottle)' : 10,
'Bottle of Wine (Mid-Range)' : 2,
'Cappuccino (regular)'  : 30,
'Meal, Inexpensive Restaurant' : 8,
'1 Pair of Jeans (Levis 501 Or Similar)' : 1,
'groceries (dollars)': 600
}

def cost_per_city_1B_Center(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += (float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (1 bedroom) in City Centre')].values[0][3]))
        result = {
        'city' : city,
        '1_Bed_Center_Total' : total
                  }
        yield result
        
def cost_per_city_3B_Center(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += round((float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (3 bedrooms) in City Centre')].values[0][3]) / 3),2)
        result = {
        'city' : city,
        '3_Bed_Center_Total' : total
                  }
        yield result

def cost_per_city_3B_Outside(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += round(((float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (3 bedrooms) Outside of Centre')].values[0][3])) / 3),2)
        result = {
        'city' : city,
        '3_Bed_Outside_Total' : total
                  }
        yield result

def cost_per_city_1B_Outside(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += (float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (1 bedroom) Outside of Centre')].values[0][3]))
        result = {
        'city' : city,
        '1_Bed_Outside_Total' : total
                  }
        yield result

In [241]:
def find_cost_of_living(data, monthly_constants):
    B1_center = pd.DataFrame(list(cost_per_city_1B_Center(data, monthly_constants)))
    B3_center = pd.DataFrame(list(cost_per_city_3B_Center(data, monthly_constants)))
    B1_Outside = pd.DataFrame(list(cost_per_city_1B_Outside(data, monthly_constants)))
    B3_Outside = pd.DataFrame(list(cost_per_city_3B_Outside(data, monthly_constants)))
    B1_center.set_index('city', inplace=True)
    B3_center.set_index('city', inplace=True)
    B1_Outside.set_index('city', inplace=True)
    B3_Outside.set_index('city', inplace=True)
    dfs = [B1_center, B3_center, B1_Outside, B3_Outside]
    city_matrix = pd.concat(dfs, axis=1)
    city_matrix['city'] = city_matrix.index
    city_matrix.drop(columns='city', inplace=True)
    return city_matrix

In [243]:
monthly_constants2 = {
'Cinema, International Release, 1 Seat' : 3,
'Fitness Club, Monthly Fee for 1 Adult' : 0,
'Basic (Electricity, Heating, Cooling, Water, Garbage) for 915 sq ft Apartment' : .5,
'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)' : .5,
'Gasoline (1 gallon)' : 15,
'Imported Beer (11.2 oz small bottle)' : 25,
'Bottle of Wine (Mid-Range)' : 3,
'Cappuccino (regular)'  : 15,
'Meal, Inexpensive Restaurant' : 20,
'1 Pair of Jeans (Levis 501 Or Similar)' : 10,
'groceries (dollars)': 200
}

In [245]:
find_cost_of_living(data, monthly_constants2)

,1_Bed_Center_Total,3_Bed_Center_Total,1_Bed_Outside_Total,3_Bed_Outside_Total
city,,,,
Palo-Alto,4415.755,3155.825,4122.485,2900.265
San-Jose,3819.940,2606.570,3562.260,2521.090
Portland,2934.115,2365.775,2555.105,2049.675
Austin,2984.135,2303.925,2413.815,1957.895
Denver,2964.960,2269.020,2631.530,2039.630
San-Diego,3307.575,2503.265,2901.165,2281.845
Washington D.C.,3641.510,2825.480,3188.170,2438.230
Boston,3913.585,2945.425,3228.535,2408.555
San-Francisco,4993.135,3515.985,4239.605,3038.635
